In [1]:
using Revise
using GLMakie
include("EFITModule/EFIT.jl")
using NPZ

In [2]:
using ParallelStencil
using ParallelStencil.FiniteDifferences3D
using GLMakie

ParallelStencil.@reset_parallel_stencil()
USE_GPU = false
@static if USE_GPU
    @init_parallel_stencil(CUDA, Float32, 3)
else
    @init_parallel_stencil(Threads, Float32, 3)
end

In [14]:
matGrid = npzread("MeshFiles/IIWMini225.npy")[:,:,1:160].+1
#Pad edges
th=2
matGrid[1:th,:,:] .=1
matGrid[end:end-th,:,:] .=1
matGrid[:,1:th,:].=1
matGrid[:,end:end-th,:].=1
matGrid[:,:,1:th].=1
matGrid[:,:,end:end-th].=1

#Sound speed in the model
cMax = 5918
cMin = 3225
#Frequency, hz
f0=2.25e6

dx = cMin/(8*f0)
dt = dx/(cMax*sqrt(3))


1.7479202979014526e-8

In [15]:
display(volume(matGrid.-1))

GLMakie.Screen(...)

In [16]:
materials = [Main.EFIT.IsoMat(0,0,1e-10), Main.EFIT.IsoMats["AISI1018"]]

2-element Vector{Main.EFIT.IsoMat}:
 Main.EFIT.IsoMat(1.0f-10, 0.0f0, 0.0f0, 0.0f0)
 Main.EFIT.IsoMat(7870.0f0, 1.11923f11, 8.1852916f10, 2.7562885f11)

In [17]:
grid = Main.EFIT.EFITGrid(Int16.(matGrid),materials,dt,dx);

sourcePositions = [(0.001,0.11,0.0125)]
sIdx = 1

pos = round.(Integer,sourcePositions[sIdx]./dx)

nframes = 1500
framerate = 30

tIterator = 0:grid.dt:grid.dt*nframes
aValues = Array{Float32,1}()
push!(aValues,0.0)

fig = Figure(resolution=(1920,1080))
lscene = LScene(fig[1,1])
fig[1,2] = Axis(fig[1,2],title="A-Scan", ylabel="Velocity Magnitude",xlabel="Iterations")
plt = volume!(lscene, Array(grid.vx),algorithm=:mip,colorrange = (0.0001, 0.1),colormap=:curl, transparency=true)
ascan = lines!(fig[1,2],zeros(size(tIterator,1)).+0.1)

#Step the simulation
function stepSim(t)
    #Isotropic step function
    Main.EFIT.SimStep!(grid)

    Main.EFIT.applyShapedSource!(grid, pos, (2,3),t,sourceParams=(f0),shape=(10,10),θ=-36,ϕ=0,c=cMax)
    velMag = sqrt.(grid.vx.^2 .+ grid.vy.^2 .+ grid.vz.^2)
    plt.volume = velMag
    push!(aValues,velMag[pos[1],pos[2],pos[3]])
    ascan[1] = aValues

end

record(stepSim, fig, "color_animation.mp4", tIterator; framerate = 30)